# Hybrid Reaction Modelling / Representation of Chemical Reactions as T-Box Graphs and Relational databases

## Abstract
Chemical reactions are fundamental processes in chemistry, and their accurate representation is crucial for various applications, including drug discovery, materials science, and chemical engineering. This paper introduces a novel hybrid approach to representing chemical reactions as T-Box graphs and relational databases, enabling more efficient querying and reasoning over chemical knowledge.
 The T-Box of the ontology should contain the high-level information of the reaction and reactands and the relational database the factual information (like molecular mass, structure, physical properties etc.). 

```
┌─────────────────────────────────────────────────┐
│           Application Layer (Python)            │
│  - Query Interface                              │
│  - Reasoning Engine                             │
│  - Data Integration Logic                       │
└───────────┬─────────────────────┬───────────────┘
            │                     │
            ▼                     ▼
┌───────────────────────┐  ┌──────────────────────┐
│   T-Box (Ontology)    │  │  A-Box + Facts (DB)  │
│                       │  │                      │
│ - Reaction classes    │  │ - Molecular mass     │
│ - Reactant types      │  │ - Structure (SMILES) │
│ - Relationships       │  │ - Properties         │
│ - Constraints         │  │ - Measurements       │
│ - Inference rules     │  │ - Concentrations     │
└───────────────────────┘  └──────────────────────┘
```

RDKit can be used to compute molecular properties and descriptors, which can then be stored in the relational database for efficient querying. The ontology can define classes and properties for different types of reactions and reactants, while the database can store specific instances and their associated data.

In a final implementation, a SPARQL query should be possible to retrieve reactions based on specific criteria, such as reactant properties or reaction conditions, leveraging both the ontology and the relational database.
(for that an SPARQL agent needs to be implemented that can query both the ontology and the database and combine the results).


## Competency Questions

### Reaction Classification and Identification
1. Determine the type of reaction (e.g., redox, acid-base, catalytic, ...) based on reactants and products.
1. Identify all reactions involving a specific molecule as a reactant or product.
1. Retrieve the stoichiometry of reactants and products for a given reaction.
1. What is the catalytic agent for a specific reaction?

### Reaction Mechanism and Dynamics
1. What are the key steps in the reaction mechanism?
1. How do changes in conditions (e.g., temperature, pressure) affect the reaction rate?
1. Which co-factors are necessary for the reaction to proceed?
1. What are the intermediate species formed during the reaction?
1. What are the electronic states of the reactants, intermediates and products during the reaction?
1. How are charges changed during the reaction?
1. How are electrons transferred during the reaction?
1. Which structural features of the catalyst interact with the reactants to facilitate the reaction?
1. What are the thermodynamic properties (e.g., enthalpy, entropy) associated with the reaction?
1. What is the activation energy of the reaction?

### Reaction Context and Environment
1. In which solvent does the reaction occur?
1. What is the role of the solvent in the reaction?


In [14]:
from owlready2 import *
from sqlmodel import Field, Session, SQLModel, create_engine, select, Relationship
from typing import Optional, List
from enum import Enum
import json

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Lipinski, rdMolDescriptors
from rdkit.Chem import Draw, rdChemReactions
from rdkit.Chem.AllChem import GetFormalCharge
import numpy as np

In [15]:
## ========================================
# PART 1: T-BOX (Ontology - High-Level Knowledge)
# ========================================

onto = get_ontology("http://biochem.org/reactions.owl")

with onto:
    # ===== Core Classes =====
    class ChemicalEntity(Thing): 
        """Base class for all chemical entities"""
        pass
    
    class Molecule(ChemicalEntity): 
        """Represents chemical molecules"""
        pass
    
    class Catalyst(ChemicalEntity):
        """Catalytic agents"""
        pass
    
    class Enzyme(Catalyst):
        """Biological catalysts"""
        pass
    
    class Solvent(ChemicalEntity):
        """Reaction medium"""
        pass
    
    # ===== Reaction Hierarchy =====
    class Reaction(Thing): 
        """Base reaction class"""
        pass
    
    class RedoxReaction(Reaction):
        """Reactions involving electron transfer"""
        pass
    
    class ElectronTransferReaction(RedoxReaction):
        """Specific electron transfer reactions"""
        pass
    
    class CatalyticReaction(Reaction):
        """Reactions requiring catalysts"""
        pass
    
    class EnzymaticReaction(CatalyticReaction):
        """Enzyme-catalyzed reactions"""
        pass
    
    class AcidBaseReaction(Reaction):
        """Proton transfer reactions"""
        pass
    
    class SubstitutionReaction(Reaction):
        """Reactions where one group replaces another"""
        pass
    
    # ===== Reaction Components =====
    class ReactionIntermediate(ChemicalEntity):
        """Transient species in reaction mechanism"""
        pass
    
    class TransitionState(Thing):
        """High-energy state between reactants and products"""
        pass
    
    # ===== Object Properties (Relationships) =====
    class hasReactant(ObjectProperty):
        domain = [Reaction]
        range = [Molecule]
    
    class hasProduct(ObjectProperty):
        domain = [Reaction]
        range = [Molecule]
    
    class catalyzedBy(ObjectProperty):
        domain = [CatalyticReaction]
        range = [Catalyst]
    
    class requiresCofactor(ObjectProperty):
        domain = [Reaction]
        range = [Molecule]
    
    class occursInSolvent(ObjectProperty):
        domain = [Reaction]
        range = [Solvent]
    
    class hasIntermediate(ObjectProperty):
        domain = [Reaction]
        range = [ReactionIntermediate]
    
    class hasTransitionState(ObjectProperty):
        domain = [Reaction]
        range = [TransitionState]
    
    class involvesMolecule(ObjectProperty):
        domain = [Reaction]
        range = [Molecule]
    
    # ===== Data Properties (Attributes) =====
    class hasStoichiometry(DataProperty, FunctionalProperty):
        domain = [Molecule]
        range = [float]
    
    class hasReactionOrder(DataProperty, FunctionalProperty):
        domain = [Reaction]
        range = [int]
    
    class isReversible(DataProperty, FunctionalProperty):
        domain = [Reaction]
        range = [bool]
    
    class hasMechanismType(DataProperty, FunctionalProperty):
        domain = [Reaction]
        range = [str]
    
    # ===== Constraints & Rules =====
    class requiresElectronDonor(ObjectProperty):
        domain = [RedoxReaction]
        range = [Molecule]
    
    class requiresElectronAcceptor(ObjectProperty):
        domain = [RedoxReaction]
        range = [Molecule]

    

In [16]:
with onto:
    class EnzymaticReaction(Reaction):
        pass
    
    atp_hydrolysis = EnzymaticReaction("ATP_Hydrolysis")
    atp = Molecule("ATP")
    atp_hydrolysis.hasReactant.append(atp)
    atp.hasStoichiometry = 1.0

In [17]:
with onto:
    class EnzymaticReaction(Reaction):
        pass
    
    atp_hydrolysis = EnzymaticReaction("Transaminase_Reaction")
    pyruvate = Molecule("Pyruvate")
    alanine = Molecule("Alanine")
    atp_hydrolysis.hasReactant.append(pyruvate)
    pyruvate.hasStoichiometry = 1.0
    atp_hydrolysis.hasReactant.append(alanine)
    alanine.hasStoichiometry = 1.0

In [18]:
# list all reactions in the ontology

for reaction in onto.Reaction.instances():
    print(f"Reaction: {reaction.name}")
    for reactant in reaction.hasReactant:
        print(f"  Reactant: {reactant.name}, Stoichiometry: {reactant.hasStoichiometry}")

Reaction: ATP_Hydrolysis
  Reactant: ATP, Stoichiometry: 1.0
  Reactant: ATP, Stoichiometry: 1.0
  Reactant: Water, Stoichiometry: None
  Reactant: ATP, Stoichiometry: 1.0
Reaction: Transamination_3FCR
  Reactant: Alanine, Stoichiometry: 1.0
  Reactant: Pyruvate, Stoichiometry: 1.0
Reaction: Transaminase_Reaction
  Reactant: Pyruvate, Stoichiometry: 1.0
  Reactant: Alanine, Stoichiometry: 1.0
  Reactant: Pyruvate, Stoichiometry: 1.0
  Reactant: Alanine, Stoichiometry: 1.0


In [19]:
# ========================================
# PART 2: A-BOX (Database - Factual Data)
# ========================================

# Database Models
class MoleculeDB(SQLModel, table=True):
    """Physical and structural properties of molecules"""
    __tablename__ = "molecules"
    
    id: Optional[int] = Field(default=None, primary_key=True)
    onto_iri: str = Field(unique=True, index=True)  # Link to ontology
    name: str = Field(index=True)
    smiles: Optional[str] = Field(default=None)
    inchi: Optional[str] = Field(default=None)
    molecular_formula: Optional[str] = Field(default=None)
    molecular_mass: Optional[float] = Field(default=None)
    
    # Physical properties
    melting_point: Optional[float] = Field(default=None)  # Kelvin
    boiling_point: Optional[float] = Field(default=None)  # Kelvin
    density: Optional[float] = Field(default=None)  # g/cm³
    solubility: Optional[str] = Field(default=None)  # JSON: {solvent: value}
    
    # Electronic properties
    charge: Optional[int] = Field(default=0)
    dipole_moment: Optional[float] = Field(default=None)  # Debye
    
    # Relationships
    reaction_reactants: List["ReactionMoleculeDB"] = Relationship(
        back_populates="molecule",
        sa_relationship_kwargs={"foreign_keys": "[ReactionMoleculeDB.molecule_id]"}
    )


class ReactionDB(SQLModel, table=True):
    """Factual data about reactions"""
    __tablename__ = "reactions"
    
    id: Optional[int] = Field(default=None, primary_key=True)
    onto_iri: str = Field(unique=True, index=True)  # Link to ontology
    name: str = Field(index=True)
    
    # Thermodynamic properties
    enthalpy: Optional[float] = Field(default=None)  # kJ/mol
    entropy: Optional[float] = Field(default=None)  # J/(mol·K)
    gibbs_free_energy: Optional[float] = Field(default=None)  # kJ/mol
    activation_energy: Optional[float] = Field(default=None)  # kJ/mol
    
    # Kinetic properties
    rate_constant: Optional[float] = Field(default=None)
    rate_constant_units: Optional[str] = Field(default=None)
    pre_exponential_factor: Optional[float] = Field(default=None)
    
    # Conditions
    temperature: Optional[float] = Field(default=None)  # Kelvin
    pressure: Optional[float] = Field(default=None)  # atm
    ph: Optional[float] = Field(default=None)
    
    # Catalyst info
    catalyst_onto_iri: Optional[str] = Field(default=None)
    catalyst_concentration: Optional[float] = Field(default=None)
    
    # Solvent info
    solvent_onto_iri: Optional[str] = Field(default=None)
    
    # Measurements
    experimental_data: Optional[str] = Field(default=None)  # JSON
    
    # Relationships
    reaction_molecules: List["ReactionMoleculeDB"] = Relationship(
        back_populates="reaction"
    )


class ReactionMoleculeDB(SQLModel, table=True):
    """Junction table for reaction-molecule relationships"""
    __tablename__ = "reaction_molecules"
    
    id: Optional[int] = Field(default=None, primary_key=True)
    reaction_id: int = Field(foreign_key="reactions.id")
    molecule_id: int = Field(foreign_key="molecules.id")
    
    role: str  # "reactant", "product", "intermediate", "cofactor"
    stoichiometry: float = Field(default=1.0)
    concentration: Optional[float] = Field(default=None)  # mol/L
    
    # Relationships
    reaction: ReactionDB = Relationship(back_populates="reaction_molecules")
    molecule: MoleculeDB = Relationship(back_populates="reaction_reactants")


class ElectronicStateDB(SQLModel, table=True):
    """Electronic states during reactions"""
    __tablename__ = "electronic_states"
    
    id: Optional[int] = Field(default=None, primary_key=True)
    molecule_id: int = Field(foreign_key="molecules.id")
    reaction_id: int = Field(foreign_key="reactions.id")
    
    state_type: str  # "ground", "excited", "radical", "ionic"
    energy: Optional[float] = Field(default=None)  # eV
    electron_configuration: Optional[str] = Field(default=None)
    spin_multiplicity: Optional[int] = Field(default=None)


/lair/source/projects/science/opensourceLab/ScientificData/ontologies/openscienceontology/chemistry/.venv/lib/python3.13/site-packages/sqlmodel/main.py:644: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.MoleculeDB, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


InvalidRequestError: Table 'molecules' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
# ========================================
# PART 3: Hybrid Query Interface
# ========================================

class HybridReactionSystem:
    """Unified interface for querying ontology and database"""
    
    def __init__(self, ontology, db_url="sqlite:///reactions.db"):
        self.onto = ontology
        self.engine = create_engine(db_url)
        SQLModel.metadata.create_all(self.engine)
    
    def add_molecule(self, name: str, onto_class=None, **properties):
        """Add molecule to both ontology and database"""
        # Create in ontology
        if onto_class is None:
            onto_class = self.onto.Molecule
        mol_onto = onto_class(name)
        
        # Create in database
        with Session(self.engine) as session:
            mol_db = MoleculeDB(
                onto_iri=mol_onto.iri,
                name=name,
                **properties
            )
            session.add(mol_db)
            session.commit()
            session.refresh(mol_db)
        
        return mol_onto, mol_db
    
    def add_reaction(self, name: str, onto_class=None, 
                     reactants=None, products=None, **properties):
        """Add reaction to both ontology and database"""
        # Create in ontology
        if onto_class is None:
            onto_class = self.onto.Reaction
        reaction_onto = onto_class(name)
        
        # Link reactants and products in ontology
        if reactants:
            for r in reactants:
                reaction_onto.hasReactant.append(r[0])  # r[0] is onto object
        if products:
            for p in products:
                reaction_onto.hasProduct.append(p[0])
        
        # Create in database
        with Session(self.engine) as session:
            reaction_db = ReactionDB(
                onto_iri=reaction_onto.iri,
                name=name,
                **properties
            )
            session.add(reaction_db)
            session.commit()
            session.refresh(reaction_db)
            
            # Add reactant/product relationships
            if reactants:
                for mol_onto, mol_db, stoich in reactants:
                    rel = ReactionMoleculeDB(
                        reaction_id=reaction_db.id,
                        molecule_id=mol_db.id,
                        role="reactant",
                        stoichiometry=stoich
                    )
                    session.add(rel)
            if products:
                for mol_onto, mol_db, stoich in products:
                    rel = ReactionMoleculeDB(
                        reaction_id=reaction_db.id,
                        molecule_id=mol_db.id,
                        role="product",
                        stoichiometry=stoich
                    )
                    session.add(rel)
            session.commit()
        
        return reaction_onto, reaction_db
    
    def query_reactions_by_type(self, reaction_class):
        """Query reactions by ontology class"""
        reactions = []
        for r in reaction_class.instances():
            with Session(self.engine) as session:
                statement = select(ReactionDB).where(
                    ReactionDB.onto_iri == r.iri
                )
                reaction_db = session.exec(statement).first()
                if reaction_db:
                    reactions.append((r, reaction_db))
        return reactions
    
    def query_molecule_properties(self, molecule_name: str):
        """Get both ontological and physical properties"""
        # Ontology data
        mol_onto = self.onto.search_one(iri=f"*{molecule_name}")
        
        # Database data
        with Session(self.engine) as session:
            if mol_onto:
                statement = select(MoleculeDB).where(
                    MoleculeDB.onto_iri == mol_onto.iri
                )
            else:
                statement = select(MoleculeDB).where(
                    MoleculeDB.name == molecule_name
                )
            mol_db = session.exec(statement).first()
        
        return {
            "ontology": mol_onto,
            "properties": mol_db
        }
    
    def query_reaction_mechanism(self, reaction_name: str):
        """Get complete reaction information"""
        # Find in ontology
        reaction_onto = self.onto.search_one(iri=f"*{reaction_name}")
        
        if not reaction_onto:
            return None
        
        # Get database info
        with Session(self.engine) as session:
            statement = select(ReactionDB).where(
                ReactionDB.onto_iri == reaction_onto.iri
            )
            reaction_db = session.exec(statement).first()
            
            # Get all molecules involved
            mol_statement = select(ReactionMoleculeDB).where(
                ReactionMoleculeDB.reaction_id == reaction_db.id
            )
            reaction_molecules = session.exec(mol_statement).all()
            
            # Fetch molecule details
            reactants = []
            products = []
            for rm in reaction_molecules:
                mol = session.get(MoleculeDB, rm.molecule_id)
                mol_info = {
                    "name": mol.name,
                    "stoichiometry": rm.stoichiometry,
                    "molecular_mass": mol.molecular_mass,
                    "smiles": mol.smiles
                }
                if rm.role == "reactant":
                    reactants.append(mol_info)
                elif rm.role == "product":
                    products.append(mol_info)
            
            return {
                "name": reaction_db.name,
                "type": type(reaction_onto).__name__,
                "reactants": reactants,
                "products": products,
                "activation_energy": reaction_db.activation_energy,
                "enthalpy": reaction_db.enthalpy,
                "temperature": reaction_db.temperature,
                "catalyst": reaction_onto.catalyzedBy,
                "intermediates": reaction_onto.hasIntermediate
            }


In [20]:
# ========================================
# PART 4: Example Usage
# ========================================

# Initialize the hybrid system
hybrid_system = HybridReactionSystem(onto)

# Add molecules with physical properties
atp_onto, atp_db = hybrid_system.add_molecule(
    "ATP",
    onto_class=onto.Molecule,
    smiles="C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)COP(=O)(O)OP(=O)(O)OP(=O)(O)O)O)O)N",
    molecular_formula="C10H16N5O13P3",
    molecular_mass=507.18,
    charge=-4
)

adp_onto, adp_db = hybrid_system.add_molecule(
    "ADP",
    onto_class=onto.Molecule,
    smiles="C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)COP(=O)(O)OP(=O)(O)O)O)O)N",
    molecular_formula="C10H15N5O10P2",
    molecular_mass=427.20,
    charge=-3
)

phosphate_onto, phosphate_db = hybrid_system.add_molecule(
    "Phosphate",
    onto_class=onto.Molecule,
    molecular_formula="PO4",
    molecular_mass=94.97,
    charge=-3
)

water_onto, water_db = hybrid_system.add_molecule(
    "Water",
    onto_class=onto.Molecule,
    smiles="O",
    molecular_formula="H2O",
    molecular_mass=18.015
)

# Add a reaction with thermodynamic data
atp_hydrolysis_onto, atp_hydrolysis_db = hybrid_system.add_reaction(
    "ATP_Hydrolysis",
    onto_class=onto.EnzymaticReaction,
    reactants=[
        (atp_onto, atp_db, 1.0),
        (water_onto, water_db, 1.0)
    ],
    products=[
        (adp_onto, adp_db, 1.0),
        (phosphate_onto, phosphate_db, 1.0)
    ],
    enthalpy=-30.5,  # kJ/mol
    gibbs_free_energy=-30.5,
    activation_energy=45.0,
    temperature=310.15,  # 37°C
    ph=7.4
)

print("✓ Data added successfully!")


✓ Data added successfully!


In [21]:
# add example for transaminase reaction with 3FCR transaminase, pyruvate and alanine

ta_3fcr_onto, ta_3fcr_db = hybrid_system.add_molecule(
    "3FCR_Transaminase",
    onto_class=onto.Enzyme,
    molecular_formula="C450H700N120O130S2",
    molecular_mass=50000.0
)

pyruvate_onto, pyruvate_db = hybrid_system.add_molecule(
    "Pyruvate",
    onto_class=onto.Molecule,
    smiles="CC(=O)C(=O)O",
    molecular_formula="C3H4O3",
    molecular_mass=88.06,
    charge=-1
)

alanine_onto, alanine_db = hybrid_system.add_molecule(
    "Alanine",
    onto_class=onto.Molecule,
    smiles="CC(C(=O)O)N",
    molecular_formula="C3H7NO2",
    molecular_mass=89.09,
    charge=0
)
ta_reaction_onto, ta_reaction_db = hybrid_system.add_reaction(
    "Transamination_3FCR",
    onto_class=onto.EnzymaticReaction,
    reactants=[(alanine_onto, alanine_db, 1.0),
               (pyruvate_onto, pyruvate_db, 1.0)],
    products=[(ta_3fcr_onto, ta_3fcr_db, 1.0)],
    enzyme=ta_3fcr_onto
)
print("✓ Transaminase reaction added successfully!")

✓ Transaminase reaction added successfully!


In [22]:
# Query examples demonstrating the hybrid approach

# 1. Query by reaction type (uses ontology)
print("=== Enzymatic Reactions ===")
enzymatic_reactions = hybrid_system.query_reactions_by_type(onto.EnzymaticReaction)
for r_onto, r_db in enzymatic_reactions:
    print(f"Reaction: {r_db.name}")
    print(f"  ΔG: {r_db.gibbs_free_energy} kJ/mol")
    print(f"  Ea: {r_db.activation_energy} kJ/mol")

# 2. Query molecule properties (hybrid)
print("\n=== ATP Properties ===")
atp_info = hybrid_system.query_molecule_properties("ATP")
print(f"Molecular Mass: {atp_info['properties'].molecular_mass} g/mol")
print(f"Charge: {atp_info['properties'].charge}")
print(f"SMILES: {atp_info['properties'].smiles}")

# 3. Query complete reaction mechanism
print("\n=== ATP Hydrolysis Mechanism ===")
mechanism = hybrid_system.query_reaction_mechanism("ATP_Hydrolysis")
print(f"Reaction Type: {mechanism['type']}")
print(f"Reactants:")
for r in mechanism['reactants']:
    print(f"  - {r['name']} (×{r['stoichiometry']}): {r['molecular_mass']} g/mol")
print(f"Products:")
for p in mechanism['products']:
    print(f"  - {p['name']} (×{p['stoichiometry']}): {p['molecular_mass']} g/mol")
print(f"ΔH: {mechanism['enthalpy']} kJ/mol")
print(f"Ea: {mechanism['activation_energy']} kJ/mol")


=== Enzymatic Reactions ===
Reaction: ATP_Hydrolysis
  ΔG: -30.5 kJ/mol
  Ea: 45.0 kJ/mol
Reaction: Transamination_3FCR
  ΔG: None kJ/mol
  Ea: None kJ/mol

=== ATP Properties ===
Molecular Mass: 507.18 g/mol
Charge: -4
SMILES: C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)COP(=O)(O)OP(=O)(O)OP(=O)(O)O)O)O)N

=== ATP Hydrolysis Mechanism ===
Reaction Type: EnzymaticReaction
Reactants:
  - ATP (×1.0): 507.18 g/mol
  - Water (×1.0): 18.015 g/mol
Products:
  - ADP (×1.0): 427.2 g/mol
  - Phosphate (×1.0): 94.97 g/mol
ΔH: -30.5 kJ/mol
Ea: 45.0 kJ/mol
